In [1]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datasets import load_dataset
from pymatgen.core import Structure
from pymatgen.util.testing import PymatgenTest
from scipy.spatial.distance import jensenshannon

from lematerial_forgebench.benchmarks.distribution_benchmark import (
    DistributionBenchmark,
)
from lematerial_forgebench.metrics.distribution_metrics import (
    MMD,
    FrechetDistance,
    JSDistance,
)
from lematerial_forgebench.preprocess.base import PreprocessorResult
from lematerial_forgebench.preprocess.distribution_preprocess import (
    DistributionPreprocessor,
)
from lematerial_forgebench.preprocess.universal_stability_preprocess import (
    UniversalStabilityPreprocessor,
)
from lematerial_forgebench.utils.distribution_utils import (
    map_space_group_to_crystal_system,
)

%load_ext autoreload
%autoreload 2

[22:15:31] WARNING  equiformer not available                                                        ]8;id=401712;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\models\registry.py\registry.py]8;;\:]8;id=265382;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\models\registry.py#141\141]8;;\

C:\Users\samue\lematerial-forgebench\.venv\Lib\site-packages\e3nn\o3\_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.


[22:15:33] INFO     Successfully registered mace model                                              ]8;id=505097;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\models\registry.py\registry.py]8;;\:]8;id=232866;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\models\registry.py#156\156]8;;\

[22:15:34] INFO     Successfully registered orb model                                               ]8;id=108800;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\models\registry.py\registry.py]8;;\:]8;id=667003;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\models\registry.py#156\156]8;;\

           INFO     Successfully registered uma model                                               ]8;id=289603;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\models\registry.py\registry.py]8;;\:]8;id=853227;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\models\registry.py#156\156]8;;\

In [2]:
with open("sample_lematbulk.pkl", "rb") as f:
    test_lemat = pickle.load(f)

In [2]:
test = PymatgenTest()

filename = "CsBr.cif"
structure = Structure.from_file(filename)
structure = structure.remove_oxidation_states()

filename2 = "CsPbBr3.cif"
structure2 = Structure.from_file(filename2)
structure2 = structure2.remove_oxidation_states()

structures = [
    # structure,
    # structure2,
    test.get_structure("Si"),
    test.get_structure("LiFePO4"),
]

C:\Users\samue\AppData\Local\Temp\ipykernel_41080\905669326.py:1: FutureWarning: PymatgenTest is deprecated, and will be removed on 2026-01-01
Use MatSciTest in pymatgen.util.testing instead.
  test = PymatgenTest()


In [5]:
stability_preprocessor = UniversalStabilityPreprocessor(model_name="uma", timeout = 100000, relax_structures = False)
stability_preprocessor_result = stability_preprocessor(structures)

ImportError: UMA/FAIRChem is not available. Please install it with: uv pip install fairchem-core>=2.1.0

In [4]:
stability_preprocessor_result.processed_structures[1].properties

{'mlip_model': 'MACECalculator',
 'model_config': 'mp',
 'energy': -191.24685668945312,
 'forces': array([[ 2.1066004e-04,  1.2175724e-04,  3.2476193e-04],
        [-1.5396724e-04,  1.8480630e-04,  9.6291013e-05],
        [-5.8492089e-05, -2.8366988e-04, -9.8216726e-05],
        [-1.0762882e-04,  6.9001362e-05,  1.4331818e-06],
        [ 4.8637316e-03,  7.9295255e-02,  4.4408597e-02],
        [-5.0997734e-03, -7.9157926e-02,  4.4897366e-02],
        [ 4.4873431e-03,  7.8111492e-02, -4.3719117e-02],
        [-4.5324862e-03, -7.9154678e-02, -4.4880651e-02],
        [ 5.9272647e-03,  9.0916738e-02, -3.7740707e-02],
        [-5.9130341e-03, -9.1406293e-02, -3.5062969e-02],
        [ 5.7322234e-03,  9.2509851e-02,  3.7506938e-02],
        [-6.7714304e-03, -9.1596551e-02,  3.3196956e-02],
        [ 1.8315822e-02,  2.1170586e-02,  8.5512251e-03],
        [-2.4236824e-02,  2.2036597e-02,  3.8901985e-02],
        [-1.9107927e-02,  1.9780591e-02, -6.7706093e-02],
        [-2.6982000e-02, -1.0208

In [32]:
distribution_preprocessor = DistributionPreprocessor()
dist_preprocessor_result = distribution_preprocessor(structures)

In [10]:
metric = JSDistance(reference_df=test_lemat) 
default_args = metric._get_compute_attributes()
metric_result = metric(preprocessor_result.processed_structures, **default_args)
print(metric_result.metrics)

FileNotFoundError: [Errno 2] No such file or directory: 'data/lematbulk_composition_counts_distribution.json'

In [11]:
metric = MMD(reference_df=test_lemat) 
default_args = metric._get_compute_attributes()
metric_result = metric(preprocessor_result.processed_structures, **default_args)
print(metric_result.metrics)

{'Volume': np.float64(0.5156492331277469), 'Density(g/cm^3)': np.float64(0.378594576755264), 'Density(atoms/A^3)': np.float64(5.477059509595428e-05), 'Average_MMD': np.float64(0.29809952682603563)}


In [14]:
metric = FrechetDistance(reference_df=test_lemat) 

sample_embeddings = list(stability_preprocessor_result.processed_structures)

default_args = metric._get_compute_attributes()
metric_result = metric(sample_embeddings, **default_args)
print(metric_result.metrics)

Index(['LeMatID', 'Volume', 'Density(g/cm^3)', 'Density(atoms/A^3)',
       'SpaceGroup', 'CrystalSystem', 'Structure', 'OrbGraphEmbeddings'],
      dtype='object')
{'FrechetDistance': 40.38935887003667}


In [34]:
final_processed_structures = []

for ind in range(0, len(dist_preprocessor_result.processed_structures)): 
    combined_structure = dist_preprocessor_result.processed_structures[ind]
    for entry in stability_preprocessor_result.processed_structures[ind].properties.keys():
        combined_structure.properties[entry] = stability_preprocessor_result.processed_structures[ind].properties[entry]
    final_processed_structures.append(combined_structure)

preprocessor_result = PreprocessorResult(processed_structures=final_processed_structures,
    config={
        "stability_preprocessor_config":stability_preprocessor_result.config,
        "distribution_preprocessor_config": dist_preprocessor_result.config,
    },
    computation_time={
        "stability_preprocessor_computation_time": stability_preprocessor_result.computation_time,
        "distribution_preprocessor_computation_time": dist_preprocessor_result.computation_time,
    },
    n_input_structures=stability_preprocessor_result.n_input_structures,
    failed_indices={
        "stability_preprocessor_failed_indices": stability_preprocessor_result.failed_indices,
        "distribution_preprocessor_failed_indices": dist_preprocessor_result.failed_indices,
    },
    warnings={
        "stability_preprocessor_warnings": stability_preprocessor_result.warnings,
        "distribution_preprocessor_warnings": dist_preprocessor_result.warnings,
    },
)

[17:03:55] WARNING  Number of input structures is less than processed + failed. This might indicate an   ]8;id=823273;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\preprocess\base.py\base.py]8;;\:]8;id=751211;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\preprocess\base.py#86\86]8;;\
                    issue.                                                                                         

In [40]:
benchmark = DistributionBenchmark(reference_df=test_lemat)
benchmark_result = benchmark.evaluate(preprocessor_result.processed_structures)

[17:06:38] ERROR    Failed to compute metric JSDistance                                            ]8;id=611874;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\evaluator.py\evaluator.py]8;;\:]8;id=839301;file://C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\evaluator.py#199\199]8;;\
                    ╭──────────────────── Traceback (most recent call last) ─────────────────────╮                 
                    │ C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\evaluator.p │                 
                    │ y:193 in evaluate                                                          │                 
                    │                                                                            │                 
                    │   190 │   │   # each metric handles its own parallelization                │                 
                    │   191 │   │   for metric_name, metric in self.metrics.items():             │                 
                    │   192 │   │   │   try:                                                     │                 
                    │ ❱ 193 │   │   │   │   result = metric.compute(                             │                 
                    │   194 │   │   │   │   │   structures=structures, **metric._get_compute_att │                 
                    │   195 │   │   │   │   )                                                    │                 
                    │   196 │   │   │   │   metric_results[metric_name] = result                 │                 
                    │                                                                            │                 
                    │ C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\metrics\dis │                 
                    │ tribution_metrics.py:151 in compute                                        │                 
                    │                                                                            │                 
                    │   148 │   │   │   │   │   dist_metrics[quant] = js                         │                 
                    │   149 │   │                                                                │                 
                    │   150 │   │   for quant in ["CompositionCounts", "Composition"]:           │                 
                    │ ❱ 151 │   │   │   js = compute_jensen_shannon_distance(                    │                 
                    │   152 │   │   │   │   reference_df,                                        │                 
                    │   153 │   │   │   │   df_all_properties,                                   │                 
                    │   154 │   │   │   │   quant,                                               │                 
                    │                                                                            │                 
                    │ C:\Users\samue\lematerial-forgebench\src\lematerial_forgebench\utils\distr │                 
                    │ ibution_utils.py:118 in compute_jensen_shannon_distance                    │                 
                    │                                                                            │                 
                    │   115 │   │   )                                                            │                 
                    │   116 │                                                                    │                 
                    │   117 │   elif crystal_param == "CompositionCounts":                       │                 
                    │ ❱ 118 │   │   with open("data/lematbulk_composition_counts_distribution.js │                 
                    │   119 │   │   │   reference_data_dist = json.load(file)                    │                 
                    │   120 │   elif crystal_param == "Composition":      

Index(['LeMatID', 'Volume', 'Density(g/cm^3)', 'Density(atoms/A^3)',
       'SpaceGroup', 'CrystalSystem', 'Structure', 'OrbGraphEmbeddings'],
      dtype='object')


In [41]:
print("JSDistance")
print(benchmark_result.evaluator_results["JSDistance"]["metric_results"]["JSDistance"].metrics)
print("Average JSDistance: " + str(benchmark_result.evaluator_results["JSDistance"]["JSDistance_value"]))
print("MMD")
print(benchmark_result.evaluator_results["MMD"]["metric_results"]["MMD"].metrics)
print("Average MMD: " + str(benchmark_result.evaluator_results["MMD"]["MMD_value"]))
print("FrechetDistance")
print(benchmark_result.evaluator_results["FrechetDistance"]["metric_results"]["FrechetDistance"].metrics)
print("Average Frechet Distance: " + str(benchmark_result.evaluator_results["FrechetDistance"]["FrechetDistance_value"]))


JSDistance
{'JSDistance': nan}
Average JSDistance: nan
MMD
{'Volume': np.float64(0.5156492331277469), 'Density(g/cm^3)': np.float64(0.378594576755264), 'Density(atoms/A^3)': np.float64(5.477059509595428e-05), 'Average_MMD': np.float64(0.29809952682603563)}
Average MMD: 0.29809952682603563
FrechetDistance
{'FrechetDistance': 40.38935887003667}
Average Frechet Distance: 40.38935887003667
